In [ ]:
!pip install stellargraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 435 kB 33.4 MB/s 


In [ ]:
# Importing Libraries
# please do go through this python notebook: 
import warnings
warnings.filterwarnings("ignore")

import csv
import pandas as pd # Pandas to create small dataframes 
import numpy as np
import datetime # Convert to unix time
import time # Convert to unix time
# If numpy is not installed already : pip3 install numpy
import numpy as np # Do aritmetic operations on arrays
# Matplotlib: used to plot graphs
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns # Plots
from matplotlib import rcParams # Size of plots  
import math
import pickle
import os
# To install xgboost: pip3 install xgboost
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

import warnings
import networkx as nx
import pdb
import pickle
from tqdm.notebook import tqdm
import os
import random
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
data_path = "gdrive/My Drive/Major Project/"

In [ ]:
df_train = pd.read_csv(data_path+"pos_train_91.csv", index_col=0)

In [ ]:
df_train.head()

,src,dest,link
496205,54500,16485,1
141147,3318,30547,1
483066,33514,4007,1
478351,28118,2641,1
505144,69985,61687,1


In [ ]:
import tensorflow

In [ ]:
from tensorflow import keras

In [ ]:
from stellargraph.mapper import (
    CorruptedGenerator,
    FullBatchNodeGenerator,
    GraphSAGENodeGenerator,
    HinSAGENodeGenerator,
    ClusterNodeGenerator,
)
from stellargraph import StellarGraph
from stellargraph.layer import GCN, DeepGraphInfomax, GraphSAGE, GAT, APPNP, HinSAGE

from stellargraph import datasets
from stellargraph.utils import plot_history

import pandas as pd
from matplotlib import pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from IPython.display import display, HTML

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras import Model

In [ ]:
train_graph = nx.from_pandas_edgelist(df_train, source='src', target='dest', create_using=nx.DiGraph())

In [ ]:
degrees = nx.degree(train_graph)

In [ ]:
degrees = dict(degrees)

In [ ]:
import collections
features = {}
for key, val in degrees.items():
  features[key] = [val]

In [ ]:
pr = nx.pagerank(train_graph, alpha=0.85)
pr = dict(pr)
for key, val in pr.items():
  features[key].append(val)

In [ ]:
katz = nx.katz_centrality(train_graph,alpha=0.005,beta=1)
katz = dict(katz)
for key, val in katz.items():
  features[key].append(val)

In [ ]:
nx.set_node_attributes(train_graph, features, "features")

In [ ]:
G = StellarGraph.from_networkx(train_graph, node_features="features")

In [ ]:
print(G.info())

StellarDiGraph: Directed multigraph
 Nodes: 75879, Edges: 452474

 Node types:
  default: [75879]
    Features: float32 vector, length 3
    Edge types: default-default->default

 Edge types:
    default-default->default: [452474]
        Weights: all 1 (default)
        Features: none


In [ ]:
fullbatch_generator = FullBatchNodeGenerator(G, sparse=True)
gcn_model = GCN(layer_sizes=[64], activations=['relu'], generator=fullbatch_generator)

corrupted_generator = CorruptedGenerator(fullbatch_generator)
gen = corrupted_generator.flow(G.nodes())

Using GCN (local pooling) filters...


In [ ]:
infomax = DeepGraphInfomax(gcn_model, corrupted_generator)
x_in, x_out = infomax.in_out_tensors()

model = Model(inputs=x_in, outputs=x_out)
model.compile(loss=tf.nn.sigmoid_cross_entropy_with_logits, optimizer=Adam(lr=1e-3))

In [ ]:
batch_size=100
epochs=50
es = EarlyStopping(monitor="loss", min_delta=0, patience=4, restore_best_weights=True,verbose=1)

In [ ]:
history = model.fit(gen, epochs=epochs, verbose=1, callbacks=[es])
model_file = open(data_path+"gcn_model.h5", "wb")
pickle.dump(model, model_file)
model_file.close()

Epoch 1/50
1/1 [==============================] - 5s 5s/step - loss: 10.0403
Epoch 2/50
1/1 [==============================] - 0s 97ms/step - loss: 8.7780
Epoch 3/50
1/1 [==============================] - 0s 95ms/step - loss: 7.6766
Epoch 4/50
1/1 [==============================] - 0s 101ms/step - loss: 6.4517
Epoch 5/50
1/1 [==============================] - 0s 99ms/step - loss: 5.1959
Epoch 6/50
1/1 [==============================] - 0s 97ms/step - loss: 3.9742
Epoch 7/50
1/1 [==============================] - 0s 96ms/step - loss: 2.7519
Epoch 8/50
1/1 [==============================] - 0s 105ms/step - loss: 1.6147
Epoch 9/50
1/1 [==============================] - 0s 96ms/step - loss: 0.6828
Epoch 10/50
1/1 [==============================] - 0s 93ms/step - loss: 2.6327
Epoch 11/50
1/1 [==============================] - 0s 93ms/step - loss: 3.5760
Epoch 12/50
1/1 [==============================] - 0s 97ms/step - loss: 3.5321
Epoch 13/50
1/1 [==============================] - 0s 100ms/

In [ ]:
x_emb_in, x_emb_out = gcn_model.in_out_tensors()

x_out = tf.squeeze(x_emb_out, axis=0)
emb_model = Model(inputs=x_emb_in, outputs=x_out)

In [ ]:
train_gen = fullbatch_generator.flow(G.nodes())
emb = emb_model.predict(train_gen)

In [ ]:
emb.shape

(75879, 64)

In [ ]:
emb_dict = {}
for i, node in enumerate(list(G.nodes())):
  emb_dict[node] = np.array(emb[i].reshape(1,64))

In [ ]:
combined = pd.read_csv(data_path+"/train_91.csv", index_col = 0)
combined.head()

,src,dest,link
73132,35080,10701,0
404654,54809,75095,0
310475,67628,60404,0
301271,57253,462,0
332230,15167,47459,1


In [ ]:
# get training embeddings
train_emb = []
for i, row in tqdm(combined.iterrows()):
  comb_emb = (emb_dict[row['src']]) * (emb_dict[row['dest']])
  train_emb.append(comb_emb[0])
train_emb = np.array(train_emb)

0it [00:00, ?it/s]

In [ ]:
train_y = combined['link']
train_y.shape

(902474,)

In [ ]:
combined_test = pd.read_csv(data_path+"/test_91.csv", index_col = 0)
combined_test.head()

,src,dest,link
487942,32198,62377,0
103900,6056,11650,1
78661,2040,22467,1
71074,143,0,1
382438,1609,5035,1


In [ ]:
# get test embeddings
test_emb = []
for i, row in tqdm(combined_test.iterrows()):
  comb_emb = (emb_dict[row['src']]) * (emb_dict[row['dest']])
  test_emb.append(comb_emb[0])

print(type(test_emb))
test_emb = np.array(test_emb)
print(test_emb.shape)
test_y = combined_test['link'] 

0it [00:00, ?it/s]

<class 'list'>
(106363, 64)


In [ ]:
train_emb_sample, _, train_y_sample, _ = train_test_split(train_emb, train_y, stratify = train_y, test_size = 0.5, random_state = 42)

In [ ]:
train_emb_sample.shape

(451237, 64)

In [ ]:
#reading
from pandas import read_hdf
df_final_train = read_hdf(data_path + 'fea_sample/storage_sample_stage4.h5', 'train_df',mode='r')
df_final_test = read_hdf(data_path + 'fea_sample/storage_sample_stage4.h5', 'test_df',mode='r')

In [ ]:
df_final_train.columns

Index(['src', 'dest', 'link', 'jaccard_followers', 'jaccard_followees',
       'cosine_followers', 'cosine_followees', 'num_followers_s',
       'num_followers_d', 'num_followees_s', 'num_followees_d',
       'inter_followers', 'inter_followees', 'adar_index', 'follows_back',
       'same_comp', 'shortest_path', 'weight_in', 'weight_out', 'weight_f1',
       'weight_f2', 'weight_f3', 'weight_f4', 'page_rank_s', 'page_rank_d',
       'katz_s', 'katz_d', 'hubs_s', 'hubs_d', 'authorities_s',
       'authorities_d', 'svd_u_s_1', 'svd_u_s_2', 'svd_u_s_3', 'svd_u_s_4',
       'svd_u_s_5', 'svd_u_s_6', 'svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3',
       'svd_u_d_4', 'svd_u_d_5', 'svd_u_d_6', 'svd_v_s_1', 'svd_v_s_2',
       'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6', 'svd_v_d_1',
       'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5', 'svd_v_d_6',
       'svd_dot_u', 'svd_dot_v', 'pref_attach_er', 'pref_attach_ee'],
      dtype='object')

In [ ]:
df_final_train.shape

(902474, 59)

In [ ]:
y_train = df_final_train.link
y_test = df_final_test.link

In [ ]:
df_final_train.drop(['src', 'dest','link'],axis=1,inplace=True)
df_final_test.drop(['src', 'dest','link'],axis=1,inplace=True)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
# GCN only 
clf = XGBClassifier(max_depth=7, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(train_emb, train_y)
train_sc = f1_score(train_y,clf.predict(train_emb))
test_sc = f1_score(test_y,clf.predict(test_emb))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.8748855826634157 test Score 0.8876855120409558


In [ ]:
# GCN only
clf = RandomForestClassifier(max_depth=12, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(train_emb, train_y)
train_sc = f1_score(train_y,clf.predict(train_emb))
test_sc = f1_score(test_y,clf.predict(test_emb))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.8769005142294911 test Score 0.8881109166180433


In [ ]:
# GCN only
from lightgbm import LGBMClassifier
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, 
                     objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                     subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, importance_type='split')
clf.fit(train_emb, train_y)
train_sc = f1_score(train_y,clf.predict(train_emb))
test_sc = f1_score(test_y,clf.predict(test_emb))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.8722291083473763 test Score 0.8878075855689176


In [ ]:
# Heuristic only
clf = XGBClassifier(max_depth=7, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.983029655226594 test Score 0.9458179808400884


In [ ]:
# Heuristic only
clf = RandomForestClassifier(max_depth=12, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9771341261464953 test Score 0.9465713006860139


In [ ]:
# Heuristic only
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, 
                     objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                     subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, importance_type='split')
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9832426574663836 test Score 0.9469947114279144


In [ ]:
df_final_train = np.hstack((df_final_train, train_emb))
print(df_final_train.shape)
df_final_test = np.hstack((df_final_test, test_emb))
print(df_final_test.shape)

(902474, 120)
(106363, 120)


In [ ]:
print(type(train_y), type(y_train))
for i in range(len(y_train.values)):
  if train_y.values[i] != y_train.values[i]: print(i)

<class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


In [ ]:
# Heuristic + GCN
clf = XGBClassifier(max_depth=7, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9832000362883642 test Score 0.945867528807096


In [ ]:
# Heuristic + GCN
clf = RandomForestClassifier(max_depth=12, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9753608172264405 test Score 0.9410945091209093


In [ ]:
# Heuristic + GCN
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, 
                     objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                     subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, importance_type='split')
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9833681626695857 test Score 0.9473258145824518
